In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
# ============================================================
# ✅ FIX PATH FOR NOTEBOOK INSIDE /notebooks
# ============================================================
import sys, os
from pathlib import Path

# Xác định thư mục cha của notebooks/
project_root = Path(os.getcwd()).parent  # => C:\Users\User\RR_model
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))  # ⚠️ thêm project_root, KHÔNG thêm src

print("📁 Project root added:", project_root)

# Kiểm tra lại
import importlib.util
print("🔍 src module found:", importlib.util.find_spec("src"))

📁 Project root added: D:\Python_code\RR_Model
🔍 src module found: ModuleSpec(name='src', loader=<_frozen_importlib_external.SourceFileLoader object at 0x0000023302A22470>, origin='D:\\Python_code\\RR_Model\\src\\__init__.py', submodule_search_locations=['D:\\Python_code\\RR_Model\\src'])


In [3]:
# ===========================================================
# 📘 Roll Rate Demo v7 - Sử dụng dữ liệu thật từ Oracle hoặc SQL file
# ===========================================================

from pathlib import Path
import sys
import pandas as pd
import numpy as np

# --- 1️⃣ Setup đường dẫn và import các module ---
root = Path(".").resolve()
sys.path.append(str(root / "src"))

from src.data_loader import load_data
from src.config import CFG, OUT_ROOT
from src.rollrate.transition import compute_transition_from_pairs, compute_transition_by_mob
#from src.rollrate.forecast import forecast_report
from src.rollrate.export_excel import export_transition_excel
from src.rollrate.backtest import (
    compute_actual_matrix,
    rollforward_validation,
    plot_matrix_diff,
    plot_distribution_compare
)

print("✅ Import modules thành công.")
print("Cấu hình CFG:", CFG)


# --- 2️⃣ Load dữ liệu thực tế ---
sql_path = r"D:\Python_code\RR_Model\sql\NTB.sql"

df = load_data(sql_path)
#df = load_data()
df['DISBURSAL_DATE'] = df['DISBURSAL_DATE'].apply(lambda x: x.replace(day=1))

#print(f"📦 Đã load {len(df):,} dòng dữ liệu từ {sql_path}")

# Kiểm tra các cột quan trọng
required_cols = [CFG["loan"], CFG["mob"], CFG["state"], CFG["ead"], CFG["cutoff"]]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"❌ Thiếu các cột cần thiết trong data: {missing}")

print("✅ Dữ liệu có đủ các cột cần thiết:", required_cols)

# Nếu chưa có PRODUCT_TYPE thì thêm mặc định
if "PRODUCT_TYPE" not in df.columns:
    df["PRODUCT_TYPE"] = "A"
    print("⚠️ Không có PRODUCT_TYPE trong data, gán mặc định = 'A'")

# Convert cutoff về datetime
df[CFG["cutoff"]] = pd.to_datetime(df[CFG["cutoff"]])





✅ Import modules thành công.
Cấu hình CFG: {'loan': 'AGREEMENT_ID', 'mob': 'MOB', 'state': 'STATE_MODEL', 'orig_date': 'DISBURSAL_DATE', 'ead': 'PRINCIPLE_OUTSTANDING', 'disb': 'DISBURSAL_AMOUNT', 'cutoff': 'CUTOFF_DATE'}
🔗 Loading data from Oracle...
=== SQL DEBUG ===
File: D:\Python_code\RR_Model\sql\NTB.sql
First 200 chars:
 select  CUTOFF_DATE, AGREEMENT_ID, trunc(DISBURSAL_DATE,'MM') DISBURSAL_DATE, DISBURSAL_AMOUNT, DPD_EOM, RISK_BUCKET, PRINCIPLE_OUTSTANDING, STATUS, MOB, MAFC_SUB_CATEGORY PRODUCT_TYPE, NPA_STAGEID, D...
Params: {}


D:\Python_code\RR_Model\src\db.py:63: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(final_sql, conn, params=bind_params)


✅ Dữ liệu có đủ các cột cần thiết: ['AGREEMENT_ID', 'MOB', 'STATE_MODEL', 'PRINCIPLE_OUTSTANDING', 'CUTOFF_DATE']


In [4]:
from src.rollrate.transition import compute_transition_by_mob
from src.rollrate.export_excel import export_transition_excel
from src.config import OUT_ROOT

print("\n=== Tính transition matrices theo Product × MOB × Score ===")
matrices_by_mob, parent_fallback = compute_transition_by_mob(df)

# Đếm real vs fallback
count_real = 0
count_fb = 0

for prod, mob_dict in matrices_by_mob.items():
    for mob, score_dict in mob_dict.items():
        for score, obj in score_dict.items():
            if obj["is_fallback"]:
                count_fb += 1
            else:
                count_real += 1

print(f"🔹 Ma trận REAL     : {count_real}")
print(f"🔸 Ma trận FALLBACK : {count_fb}")
print(f"🔢 Tổng             : {count_real + count_fb}")

# Xuất Excel
export_transition_excel(
    matrices_by_mob,
    parent_fallback,
    out_path=OUT_ROOT / "Transition_ByProduct_MOB_Score_2.xlsx"
)



=== Tính transition matrices theo Product × MOB × Score ===
⚠️ Có 3 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'SOLDOUT']
⚙️ Built parent fallback for (product=SALPIL, score=A)
⚠️ Có 3 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'SOLDOUT']
⚙️ Built parent fallback for (product=SALPIL, score=B)
⚠️ Có 3 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'SOLDOUT']
⚙️ Built parent fallback for (product=SALPIL, score=C)
⚠️ Có 3 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'SOLDOUT']
⚙️ Built parent fallback for (product=SALPIL, score=D)
⚠️ Có 7 hàng có tổng weight = 0: ['DPD30+', 'DPD60+', 'DPD90+', 'DPD120+', 'DPD180+', 'WRITEOFF', 'SOLDOUT']
⚠️ Có 7 hàng có tổng weight = 0: ['DPD60+', 'DPD90+', 'DPD120+', 'DPD180+', 'PREPAY', 'WRITEOFF', 'SOLDOUT']
⚠️ Có 6 hàng có tổng weight = 0: ['DPD90+', 'DPD120+', 'DPD180+', 'PREPAY', 'WRITEOFF', 'SOLDOUT']
⚠️ Có 5 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'WRITEOFF', 'SOLDOUT']
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 

In [5]:


export_transition_excel(
    matrices_by_mob,
    parent_fallback,
    out_path=OUT_ROOT / "Transition_ByProduct_MOB_Score_2.xlsx"
)


✅ Xuất file transition thành công → D:\Python_code\RR_Model\outputs\Transition_ByProduct_MOB_Score_2.xlsx


## kiểm tra chi tiết ma trận có hoạt động đúng không

In [6]:
from src.rollrate.transition import (
    make_pairs,
    compute_transition_from_pairs,
    compute_transition_by_mob
)

In [7]:
from src.config import (
    CFG,
    BUCKETS_CANON,
    ABSORBING_BASE,
    WEIGHT_METHOD,
    ROLL_WINDOW,
    MIN_OBS,
    MIN_EAD,
)

In [8]:
def debug_transition_group(df, debug_prod, debug_score, debug_mob):
    """
    Debug đầy đủ:
      - Raw data của nhóm theo ROLL_WINDOW
      - Weight (ead_raw, time_weight, ead_t)
      - Pairs (t → t+1)
      - Manual matrix
      - Matrix từ code
    """

    loan   = CFG["loan"]
    mob    = CFG["mob"]
    state  = CFG["state"]
    cutoff = CFG["cutoff"]

    print("\n" + "="*80)
    print(f"DEBUG GROUP ⇒ product={debug_prod}, score={debug_score}, MOB={debug_mob}")
    print("="*80)

    # --------------------------------------------------------------------------------------
    # 1) RUN make_pairs ONCE (FULL DATA) – This now returns: ead_raw, time_weight, ead_t
    # --------------------------------------------------------------------------------------
    pairs_all = make_pairs(df)

    print(f"\n🔍 Tổng số pairs toàn dataset = {len(pairs_all):,}")

    # --------------------------------------------------------------------------------------
    # 2) Lấy raw của nhóm này trong pairs
    # --------------------------------------------------------------------------------------
    pairs_slice = pairs_all[
        (pairs_all["product_t"] == debug_prod) &
        (pairs_all["score_t"] == debug_score) &
        (pairs_all["mob_t"] == debug_mob)
    ].copy()

    print(f"\n🔍 PAIRS (t → t+1) CỦA NHÓM NÀY = {len(pairs_slice):,}")

    if pairs_slice.empty:
        print("❌ Nhóm này không có bất kỳ pair nào.")
        return None

    # --------------------------------------------------------------------------------------
    # 3) SHOW DETAILED PAIRS
    # --------------------------------------------------------------------------------------
    show_cols = [
        loan,
        "mob_t", "mob_t1",
        "state_t", "state_t1",
        "ead_raw",
        "time_weight",
        "ead_t",
        "product_t", "score_t",
    ]

    print("\n=========== DỮ LIỆU PAIRS CHI TIẾT ===========")
    display(pairs_slice[show_cols].head(200))

    # --------------------------------------------------------------------------------------
    # 4) MANUAL AGGREGATION
    # --------------------------------------------------------------------------------------
    print("\n=========== MANUAL AGG (state_t → state_t1) ===========")

    manual_ct = (
        pairs_slice
        .groupby(["state_t", "state_t1"])["ead_t"]
        .sum()
        .unstack(fill_value=0)
        .reindex(index=BUCKETS_CANON + list(ABSORBING_BASE),
                 columns=BUCKETS_CANON + list(ABSORBING_BASE),
                 fill_value=0)
    )

    display(manual_ct)

    print("\n=========== MANUAL NORMALIZED MATRIX ===========")
    manual_P = manual_ct.div(manual_ct.sum(axis=1).replace(0, np.nan), axis=0).fillna(0)
    display(manual_P.round(4))

    # --------------------------------------------------------------------------------------
    # 5) MATRIX FROM CODE (compute_transition_from_pairs)
    # --------------------------------------------------------------------------------------
    print("\n=========== MATRIX FROM CODE ===========")
    code_P = compute_transition_from_pairs(
        pairs_slice,
        value_col="ead_t",
        parent_P=None,
        zero_row_policy="parent",
        alpha_smooth=0.0,
    )
    display(code_P.round(4))

    # --------------------------------------------------------------------------------------
    # 6) DIFFERENCE manual vs code
    # --------------------------------------------------------------------------------------
    print("\n=========== DIFFERENCE (manual - code) ===========")
    diff = (manual_P - code_P).round(4)
    display(diff)

    print("\n=== DONE DEBUG ===")

    return {
        "pairs": pairs_slice,
        "manual_counts": manual_ct,
        "manual_P": manual_P,
        "code_P": code_P,
        "diff": diff
    }


In [9]:
debug_prod  = "SALPIL"
debug_score = "A"
debug_mob   = 0

debug_out = debug_transition_group(df, debug_prod, debug_score, debug_mob)



DEBUG GROUP ⇒ product=SALPIL, score=A, MOB=0

🔍 Tổng số pairs toàn dataset = 1,783,607

🔍 PAIRS (t → t+1) CỦA NHÓM NÀY = 7,493

=========== DỮ LIỆU PAIRS CHI TIẾT ===========


,AGREEMENT_ID,mob_t,mob_t1,state_t,state_t1,ead_raw,time_weight,ead_t,product_t,score_t
2861370,5758469,0,1,DPD0,DPD0,75390000,0.28243,2.129236e+07,SALPIL,A
2275822,5773770,0,1,DPD0,DPD0,21540000,0.28243,6.083532e+06,SALPIL,A
3071639,5785210,0,1,DPD0,DPD0,30827125,0.28243,8.706491e+06,SALPIL,A
2285749,5787940,0,1,DPD0,DPD0,21540000,0.28243,6.083532e+06,SALPIL,A
2286042,5789176,0,1,DPD0,DPD0,34464000,0.28243,9.733652e+06,SALPIL,A
...,...,...,...,...,...,...,...,...,...,...
2305345,5819658,0,1,DPD0,DPD0,38772000,0.28243,1.095036e+07,SALPIL,A
2305397,5819794,0,1,DPD0,DPD0,64620000,0.28243,1.825060e+07,SALPIL,A
3291199,5819813,0,1,DPD0,DPD0,39849000,0.28243,1.125453e+07,SALPIL,A
2305527,5819849,0,1,DPD0,DPD0,32310000,0.28243,9.125298e+06,SALPIL,A



=========== MANUAL AGG (state_t → state_t1) ===========


state_t1,DPD0,DPD1+,DPD30+,DPD60+,DPD90+,DPD120+,DPD180+,PREPAY,WRITEOFF,SOLDOUT,WRITEOFF,PREPAY,SOLDOUT
state_t,,,,,,,,,,,,,
DPD0,1.296242e+11,1.086170e+09,0.000000e+00,0.0,0.0,0.0,0.0,1.929769e+09,0.0,0.0,0.0,1.929769e+09,0.0
DPD1+,0.000000e+00,1.390840e+07,1.126580e+07,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0
DPD30+,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0
DPD60+,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0
DPD90+,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0
DPD120+,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0
DPD180+,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0
PREPAY,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,4.595833e+08,0.0,0.0,0.0,4.595833e+08,0.0
WRITEOFF,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0



=========== MANUAL NORMALIZED MATRIX ===========


...


=========== MATRIX FROM CODE ===========
⚠️ Có 7 hàng có tổng weight = 0: ['DPD30+', 'DPD60+', 'DPD90+', 'DPD120+', 'DPD180+', 'WRITEOFF', 'SOLDOUT']


state_t1,DPD0,DPD1+,DPD30+,DPD60+,DPD90+,DPD120+,DPD180+,PREPAY,WRITEOFF,SOLDOUT
state_t,,,,,,,,,,
DPD0,0.9773,0.0082,0.0000,0.0,0.0,0.0,0.0,0.0145,0.0,0.0
DPD1+,0.0000,0.5525,0.4475,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
DPD30+,0.0000,0.0000,1.0000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
DPD60+,0.0000,0.0000,0.0000,1.0,0.0,0.0,0.0,0.0000,0.0,0.0
DPD90+,0.0000,0.0000,0.0000,0.0,1.0,0.0,0.0,0.0000,0.0,0.0
DPD120+,0.0000,0.0000,0.0000,0.0,0.0,1.0,0.0,0.0000,0.0,0.0
DPD180+,0.0000,0.0000,0.0000,0.0,0.0,0.0,1.0,0.0000,0.0,0.0
PREPAY,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,1.0000,0.0,0.0
WRITEOFF,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,1.0,0.0



=========== DIFFERENCE (manual - code) ===========


state_t1,DPD0,DPD1+,DPD120+,DPD180+,DPD30+,DPD60+,DPD90+,PREPAY,PREPAY,SOLDOUT,SOLDOUT,WRITEOFF,WRITEOFF
state_t,,,,,,,,,,,,,
DPD0,-0.014,-0.0001,0.0,0.0,0.0,0.0,0.0,-0.0002,-0.0002,0.0,0.0,0.0,0.0
DPD1+,0.000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0
DPD120+,0.000,0.0000,-1.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0
DPD180+,0.000,0.0000,0.0,-1.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0
DPD30+,0.000,0.0000,0.0,0.0,-1.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0
DPD60+,0.000,0.0000,0.0,0.0,0.0,-1.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0
DPD90+,0.000,0.0000,0.0,0.0,0.0,0.0,-1.0,0.0000,0.0000,0.0,0.0,0.0,0.0
PREPAY,0.000,0.0000,0.0,0.0,0.0,0.0,0.0,-0.5000,-0.5000,0.0,0.0,0.0,0.0
PREPAY,0.000,0.0000,0.0,0.0,0.0,0.0,0.0,-0.5000,-0.5000,0.0,0.0,0.0,0.0



=== DONE DEBUG ===


In [10]:
if debug_out:
    debug_out["pairs"].to_excel("debug_pairs_detail2.xlsx", index=False)
    print("✅ Saved → debug_pairs_detail2.xlsx")

✅ Saved → debug_pairs_detail2.xlsx


In [33]:
from src.rollrate.forecast import (
    forecast_vintage,
    forecast_to_dataframe,
    forecast_to_distribution,
    cumulative_loss_rate,
)

fc_ead = forecast_vintage(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    product="SALPIL",
    score="A",
    vintage_date=pd.to_datetime("2025-01-01"),
    max_mob=29
)

# EAD theo state
df_ead = forecast_to_dataframe(fc_ead)

# Tỷ lệ theo EAD ban đầu (%)
dist_dict = forecast_to_distribution(fc_ead)
df_dist = forecast_to_dataframe(dist_dict)

# Cumulative loss rate theo WRITEOFF trên EAD ban đầu:
cum_loss = cumulative_loss_rate(fc_ead, default_state="WRITEOFF")


In [34]:
df_dist

,DPD0,DPD1+,DPD30+,DPD60+,DPD90+,DPD120+,DPD180+,PREPAY,WRITEOFF,SOLDOUT
MOB,,,,,,,,,,
8,0.851190,0.029831,0.027016,0.024094,0.062866,0.0,0.0,0.000000,0.005003,0.0
9,0.824959,0.029664,0.023666,0.020338,0.081735,0.0,0.0,0.012587,0.007051,0.0
10,0.795213,0.035154,0.024224,0.016316,0.092455,0.0,0.0,0.023537,0.013100,0.0
11,0.765977,0.033962,0.027383,0.016413,0.100759,0.0,0.0,0.037775,0.017732,0.0
12,0.734023,0.032489,0.029468,0.016742,0.108602,0.0,0.0,0.054727,0.023948,0.0
13,0.707723,0.035013,0.025740,0.017891,0.110933,0.0,0.0,0.067786,0.034913,0.0
14,0.682139,0.036161,0.027380,0.016798,0.107452,0.0,0.0,0.078053,0.052016,0.0
15,0.659724,0.034351,0.028943,0.016425,0.104683,0.0,0.0,0.089233,0.066641,0.0
16,0.639886,0.031489,0.027964,0.016753,0.098088,0.0,0.0,0.100684,0.085137,0.0


In [12]:

# lấy 1 key bất kỳ
product, score, vintage = list(results.keys())[0]

df_one = forecast_to_dataframe(results[(product, score, vintage)])
print(df_one)


STATE_MODEL      DPD0    DPD1+    DPD30+    DPD60+    DPD90+  DPD120+  \
MOB                                                                     
30           0.016548  0.00197  0.001182  0.003152  0.043341      0.0   

STATE_MODEL  DPD180+    PREPAY  WRITEOFF  SOLDOUT  
MOB                                                
30               0.0  0.690701  0.243105      0.0  


In [13]:
def results_to_long_df(results):
    """
    Convert results từ forecast_all_vintages → long format:
    
    Columns:
        PRODUCT_TYPE
        RISK_SCORE
        VINTAGE_DATE
        MOB
        <BUCKETS_CANON columns: DPD0, DPD1+, ...>
    """
    records = []

    for (product, score, vintage_date), fc_dict in results.items():
        for mob, dist_series in fc_dict.items():

            row = {
                "PRODUCT_TYPE": product,
                "RISK_SCORE": score,
                "VINTAGE_DATE": vintage_date,
                "MOB": mob,
            }

            # dist_series = Series(BUCKETS_CANON)
            row.update(dist_series.to_dict())

            records.append(row)

    return pd.DataFrame(records)


In [14]:
df_long = results_to_long_df(results)
print(df_long.head())


  PRODUCT_TYPE RISK_SCORE VINTAGE_DATE  MOB      DPD0     DPD1+    DPD30+  \
0       SALPIL          A   2023-01-01   30  0.016548  0.001970  0.001182   
1       SALPIL          A   2023-02-01   30  0.043218  0.001995  0.002660   
2       SALPIL          A   2023-03-01   30  0.074114  0.005995  0.003815   
3       SALPIL          A   2023-04-01   29  0.074301  0.005453  0.005453   
4       SALPIL          A   2023-05-01   28  0.091582  0.004625  0.002775   

     DPD60+    DPD90+  DPD120+  DPD180+    PREPAY  WRITEOFF  SOLDOUT  
0  0.003152  0.043341      0.0      0.0  0.690701  0.243105      0.0  
1  0.002660  0.040559      0.0      0.0  0.625000  0.283910      0.0  
2  0.003815  0.038692      0.0      0.0  0.634877  0.238692      0.0  
3  0.002045  0.038855      0.0      0.0  0.627130  0.246762      0.0  
4  0.002775  0.038853      0.0      0.0  0.666975  0.192414      0.0  


In [15]:
df_long.to_excel('check_forecast.xlsx')

In [16]:
from src.rollrate.lifecycle import build_full_lifecycle

df_lifecycle = build_full_lifecycle(
    df_raw=df, 
    matrices_by_mob=matrices_by_mob, 
    max_mob=29
)

print(df_lifecycle.head())


  PRODUCT_TYPE RISK_SCORE VINTAGE_DATE  MOB      DPD0     DPD1+    DPD30+  \
0       SALPIL          A   2023-01-01    0  0.994081  0.005919  0.000000   
1       SALPIL          A   2023-01-01    1  0.974349  0.020521  0.000395   
2       SALPIL          A   2023-01-01    2  0.949487  0.026835  0.010655   
3       SALPIL          A   2023-01-01    3  0.916338  0.037096  0.015391   
4       SALPIL          A   2023-01-01    4  0.892660  0.037885  0.016575   

     DPD60+    DPD90+  DPD120+  DPD180+    PREPAY  WRITEOFF  SOLDOUT  
0  0.000000  0.000000      0.0      0.0  0.000000       0.0      0.0  
1  0.000000  0.000000      0.0      0.0  0.004736       0.0      0.0  
2  0.000000  0.000000      0.0      0.0  0.013023       0.0      0.0  
3  0.009866  0.000000      0.0      0.0  0.021310       0.0      0.0  
4  0.014601  0.007893      0.0      0.0  0.030387       0.0      0.0  


In [17]:
df_lifecycle.to_excel('check_forecast.xlsx')

In [22]:
from src.rollrate.lifecycle import add_del30_on_disb,tag_forecast_rows,export_lifecycle_with_formatting,add_del30_on_disb,aggregate_del30_to_product, make_del30_pivot, export_heatmap

In [19]:
df_lifecycle = build_full_lifecycle(df, matrices_by_mob, max_mob=29)

df_del30 = add_del30_on_disb(df_lifecycle, df)

print(df_del30[["PRODUCT_TYPE","RISK_SCORE","VINTAGE_DATE","MOB","DEL30_PCT_ON_DISB"]].head())


  PRODUCT_TYPE RISK_SCORE VINTAGE_DATE  MOB  DEL30_PCT_ON_DISB
0       SALPIL          A   2023-01-01    0           0.000000
1       SALPIL          A   2023-01-01    1           0.000395
2       SALPIL          A   2023-01-01    2           0.010655
3       SALPIL          A   2023-01-01    3           0.025257
4       SALPIL          A   2023-01-01    4           0.039069


In [20]:
df_del30.to_excel('check_forecast_del30.xlsx')

In [21]:
# 1) Build lifecycle (actual + forecast)
df_lifecycle = build_full_lifecycle(df, matrices_by_mob, max_mob=29)

# 2) Gắn cột IS_FORECAST để phân biệt history / forecast
df_lifecycle = tag_forecast_rows(df_lifecycle, df)

# 3) Xuất Excel với highlight
export_lifecycle_with_formatting(df_lifecycle, filename="lifecycle_with_highlight.xlsx")


✔ Excel exported with forecast highlighted: lifecycle_with_highlight.xlsx


In [25]:
df_lifecycle = build_full_lifecycle(df, matrices_by_mob, max_mob=29)


In [26]:
df_del30 = add_del30_on_disb(df_lifecycle, df)


In [27]:
df_product = aggregate_del30_to_product(df_del30)


In [28]:
df_pivot = make_del30_pivot(df_product)     # bản Product-level
# hoặc:
# df_pivot = make_del30_pivot(df_del30)    # bản Product × Score


In [29]:
export_heatmap(df_pivot, "DEL30P_Product_Heatmap.xlsx")


✔ Exported heatmap to DEL30P_Product_Heatmap.xlsx


In [30]:
df_pivot

MOB,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
Cohort,,,,,,,,,,,,,,,,,,,,,
2023-01-01,0.0,0.023585,0.218731,0.457370,0.665919,0.866502,0.991950,1.119916,1.212433,1.151771,...,0.942609,0.932939,0.861894,0.794977,0.777940,0.665344,0.562192,0.466282,0.388925,0.310034
2023-02-01,0.0,0.019056,0.409303,0.742089,1.069399,1.288112,1.467470,1.642379,1.729958,1.823620,...,1.187354,1.084685,0.995678,0.940708,0.874146,0.770485,0.642497,0.534999,0.443032,0.363617
2023-03-01,0.0,0.042775,0.441730,0.777272,1.014842,1.232327,1.442365,1.565327,1.624947,1.411614,...,1.097188,0.998123,0.922740,0.850664,0.794914,0.674832,0.570618,0.475125,0.409281,0.327411
2023-04-01,0.0,0.022087,0.337677,0.657395,0.916477,1.162903,1.352132,1.500528,1.395961,1.509853,...,1.030750,0.943345,0.866081,0.803249,0.704557,0.589754,0.494103,0.421707,0.337946,0.000000
2023-05-01,0.0,0.016627,0.259148,0.602224,0.883354,1.128346,1.342972,1.392185,1.504757,1.448082,...,0.878726,0.813472,0.748708,0.645177,0.590219,0.507066,0.414853,0.337977,0.269422,0.000000
2023-06-01,0.0,0.007030,0.099874,0.285384,0.440149,0.575777,0.690324,0.797409,0.837630,0.880431,...,0.586582,0.546759,0.482560,0.429492,0.386010,0.311068,0.250697,0.198838,0.159465,0.000000
2023-07-01,0.0,0.012816,0.107110,0.269736,0.428117,0.572148,0.703435,0.809751,0.854118,0.887775,...,0.601178,0.542839,0.467959,0.440704,0.402358,0.309383,0.244050,0.191081,0.152072,0.000000
2023-08-01,0.0,0.006458,0.091838,0.237626,0.391428,0.515105,0.669762,0.751951,0.800337,0.762819,...,0.568116,0.501479,0.435381,0.407208,0.348094,0.288741,0.235655,0.191646,0.156971,0.000000
2023-09-01,0.0,0.002795,0.081583,0.232080,0.386103,0.551000,0.637892,0.722612,0.719989,0.770625,...,0.508519,0.454379,0.411799,0.372382,0.336212,0.286643,0.239525,0.199388,0.166639,0.000000


In [32]:
df_del30.to_excel('disbur.xlsx')

In [35]:
from src.rollrate.forecast import forecast_all_vintages, forecast_to_dataframe


In [37]:
results = forecast_all_vintages(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=29,            # hoặc 36, 60 tuỳ bạn
    enable_macro=False
)


In [38]:
# Lấy 1 key bất kỳ (product, score, vintage)
sample_key = next(iter(results.keys()))

print("Sample segment:", sample_key)

# Kết quả dạng dict {mob → EAD vector}
sample_fc = results[sample_key]

# Chuyển thành DataFrame theo MOB
df_sample = forecast_to_dataframe(sample_fc)

df_sample


Sample segment: ('SALPIL', 'A', Timestamp('2023-01-01 00:00:00'))


STATE_MODEL,DPD0,DPD1+,DPD30+,DPD60+,DPD90+,DPD120+,DPD180+,PREPAY,WRITEOFF,SOLDOUT
MOB,,,,,,,,,,
30,245665928.0,40671968.0,34105024.0,43129825.0,492611926.0,0.0,0.0,0.0,9.012319e+09,0.0


In [39]:
from src.rollrate.forecast import forecast_to_distribution

dist_dict = forecast_to_distribution(sample_fc)
df_dist = forecast_to_dataframe(dist_dict)
df_dist


STATE_MODEL,DPD0,DPD1+,DPD30+,DPD60+,DPD90+,DPD120+,DPD180+,PREPAY,WRITEOFF,SOLDOUT
MOB,,,,,,,,,,
30,0.024894,0.004121,0.003456,0.00437,0.049918,0.0,0.0,0.0,0.913241,0.0


In [42]:
from src.rollrate.lifecycle import build_full_lifecycle_amount, tag_forecast_rows_amount

df_life = build_full_lifecycle_amount(df, matrices_by_mob, max_mob=29)

df_life2 = tag_forecast_rows_amount(df_life, df)

df_life2.head()


,PRODUCT_TYPE,RISK_SCORE,VINTAGE_DATE,MOB,DPD0,DPD1+,DPD30+,DPD60+,DPD90+,DPD120+,DPD180+,PREPAY,WRITEOFF,SOLDOUT,ACTUAL_MAX_MOB,IS_FORECAST
0,SALPIL,A,2023-01-01,0,5.270872e+10,3.173485e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30,0
1,SALPIL,A,2023-01-01,1,5.014614e+10,1.089986e+09,21540000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30,0
2,SALPIL,A,2023-01-01,2,4.718645e+10,1.359809e+09,577088071.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30,0
3,SALPIL,A,2023-01-01,3,4.400362e+10,1.817597e+09,791155878.0,534143803.0,0.0,0.0,0.0,0.0,0.0,0.0,30,0
4,SALPIL,A,2023-01-01,4,4.124273e+10,1.854710e+09,830096726.0,752995981.0,436883942.0,0.0,0.0,0.0,0.0,0.0,30,0


In [43]:
results

{('SALPIL',
  'A',
  Timestamp('2023-01-01 00:00:00')): {30: STATE_MODEL
  DPD0        2.456659e+08
  DPD1+       4.067197e+07
  DPD30+      3.410502e+07
  DPD60+      4.312982e+07
  DPD90+      4.926119e+08
  DPD120+     0.000000e+00
  DPD180+     0.000000e+00
  PREPAY      0.000000e+00
  WRITEOFF    9.012319e+09
  SOLDOUT     0.000000e+00
  Name: PRINCIPLE_OUTSTANDING, dtype: float64},
 ('SALPIL',
  'A',
  Timestamp('2023-02-01 00:00:00')): {30: STATE_MODEL
  DPD0        5.393636e+08
  DPD1+       3.024251e+07
  DPD30+      3.306927e+07
  DPD60+      3.639697e+07
  DPD90+      5.038280e+08
  DPD120+     0.000000e+00
  DPD180+     0.000000e+00
  PREPAY      0.000000e+00
  WRITEOFF    8.075195e+09
  SOLDOUT     0.000000e+00
  Name: PRINCIPLE_OUTSTANDING, dtype: float64},
 ('SALPIL',
  'A',
  Timestamp('2023-03-01 00:00:00')): {30: STATE_MODEL
  DPD0        1.225426e+09
  DPD1+       6.263988e+07
  DPD30+      5.545425e+07
  DPD60+      6.519357e+07
  DPD90+      5.710500e+08
  DPD120+ 

In [49]:
from src.rollrate.lifecycle import (
    build_full_lifecycle_amount,
    tag_forecast_rows_amount,
    add_del_metrics,
    aggregate_to_product,
    make_metric_pivot,
get_actual_all_vintages_amount
)

In [45]:


# 1. Build lifecycle amount-based
df_life = build_full_lifecycle_amount(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=29
)

# 2. Tag forecast rows
df_life = tag_forecast_rows_amount(df_life, df)

# 3. Add DEL30/60/90
df_del = add_del_metrics(df_life, df)

# 4. Aggregate lên PRODUCT
df_prod = aggregate_to_product(df_del)

# 5. Pivot từng metric
pivot_30 = make_metric_pivot(df_prod, "DEL30_PCT")
pivot_60 = make_metric_pivot(df_prod, "DEL60_PCT")
pivot_90 = make_metric_pivot(df_prod, "DEL90_PCT")

pivot_30.head()


D:\Python_code\RR_Model\src\rollrate\lifecycle.py:213: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  


MOB,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
VINTAGE_DATE,,,,,,,,,,,,,,,,,,,,,
2023-01-01,0.0,0.000005,0.000043,0.000089,0.000127,0.000163,0.000184,0.000205,0.000219,0.000201,...,0.000101,0.000095,0.000079,0.000063,0.000054,0.000044,0.000033,0.000026,0.000020,0.000014
2023-02-01,0.0,0.000003,0.000067,0.000119,0.000168,0.000200,0.000225,0.000248,0.000260,0.000263,...,0.000119,0.000100,0.000084,0.000073,0.000060,0.000051,0.000041,0.000032,0.000025,0.000019
2023-03-01,0.0,0.000009,0.000075,0.000128,0.000164,0.000197,0.000228,0.000245,0.000239,0.000195,...,0.000109,0.000092,0.000079,0.000068,0.000057,0.000047,0.000038,0.000030,0.000024,0.000018
2023-04-01,0.0,0.000004,0.000063,0.000119,0.000163,0.000206,0.000237,0.000255,0.000224,0.000241,...,0.000115,0.000097,0.000083,0.000072,0.000057,0.000046,0.000036,0.000029,0.000021,0.000000
2023-05-01,0.0,0.000004,0.000057,0.000129,0.000184,0.000234,0.000273,0.000274,0.000294,0.000265,...,0.000114,0.000098,0.000086,0.000068,0.000056,0.000047,0.000038,0.000028,0.000023,0.000000


In [48]:
df_life.to_excel('df_life.xlsx')

In [47]:
df_life

,PRODUCT_TYPE,RISK_SCORE,VINTAGE_DATE,MOB,DPD0,DPD1+,DPD30+,DPD60+,DPD90+,DPD120+,DPD180+,PREPAY,WRITEOFF,SOLDOUT,ACTUAL_MAX_MOB,IS_FORECAST
0,SALPIL,A,2023-01-01,0,5.270872e+10,3.173485e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,30,0
1,SALPIL,A,2023-01-01,1,5.014614e+10,1.089986e+09,2.154000e+07,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,30,0
2,SALPIL,A,2023-01-01,2,4.718645e+10,1.359809e+09,5.770881e+08,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,30,0
3,SALPIL,A,2023-01-01,3,4.400362e+10,1.817597e+09,7.911559e+08,5.341438e+08,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,30,0
4,SALPIL,A,2023-01-01,4,4.124273e+10,1.854710e+09,8.300967e+08,7.529960e+08,4.368839e+08,0.0,0.0,0.000000e+00,0.000000e+00,0.0,30,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3967,SALPIL,D,2025-09-01,25,2.414409e+08,3.230099e+07,3.410893e+07,1.830331e+07,7.515272e+07,0.0,0.0,3.177701e+08,1.116516e+09,0.0,0,1
3968,SALPIL,D,2025-09-01,26,2.250916e+08,2.907312e+07,3.562325e+07,1.722274e+07,6.796198e+07,0.0,0.0,3.264682e+08,1.134152e+09,0.0,0,1
3969,SALPIL,D,2025-09-01,27,2.155207e+08,3.143634e+07,2.469967e+07,1.758702e+07,6.410590e+07,0.0,0.0,3.345719e+08,1.147671e+09,0.0,0,1
3970,SALPIL,D,2025-09-01,28,2.065709e+08,2.808826e+07,2.911303e+07,1.035628e+07,5.980187e+07,0.0,0.0,3.371530e+08,1.164509e+09,0.0,0,1


In [52]:
from src.rollrate.lifecycle import get_actual_all_vintages_amount

actual_results = get_actual_all_vintages_amount(df)



In [53]:
key = list(actual_results.keys())[0]
mob_dict = actual_results[key]
df_actual_cohort = pd.DataFrame(mob_dict).T


In [56]:
product, score, vintage_date = key
df_vintage = df[(df["PRODUCT_TYPE"] == product) &
                    (df["RISK_SCORE"] == score) &
                    (df[CFG["orig_date"]] == vintage_date)]

mob_check = 1
df_m = df_vintage[df_vintage[CFG["mob"]] == mob_check]

check_raw = df_m.groupby(CFG["state"])[CFG["ead"]].sum().reindex(BUCKETS_CANON, fill_value=0)
actual_vec = mob_dict.get(mob_check)

check_raw, actual_vec


(STATE_MODEL
 DPD0        50146138244
 DPD1+        1089985737
 DPD30+         21540000
 DPD60+                0
 DPD90+                0
 DPD120+               0
 DPD180+               0
 PREPAY                0
 WRITEOFF              0
 SOLDOUT               0
 Name: PRINCIPLE_OUTSTANDING, dtype: int64,
 STATE_MODEL
 DPD0        5.014614e+10
 DPD1+       1.089986e+09
 DPD30+      2.154000e+07
 DPD60+      0.000000e+00
 DPD90+      0.000000e+00
 DPD120+     0.000000e+00
 DPD180+     0.000000e+00
 PREPAY      0.000000e+00
 WRITEOFF    0.000000e+00
 SOLDOUT     0.000000e+00
 Name: PRINCIPLE_OUTSTANDING, dtype: float64)

In [58]:
from src.rollrate.lifecycle import get_actual_all_vintages_amount
from src.config import CFG, BUCKETS_CANON

# --- THÔNG SỐ COHORT BẠN MUỐN KIỂM TRA ---
product = "SALPIL"
score = "A"
vintage_date = pd.Timestamp("2023-01-01")
mob_check = 1

orig_col = CFG["orig_date"]
mob_col  = CFG["mob"]
state_col = CFG["state"]
ead_col  = CFG["ead"]


# ==========================================================
# 1) RAW DATA CHI TIẾT (theo hợp đồng)
# ==========================================================

df_raw_group = df[
    (df["PRODUCT_TYPE"] == product) &
    (df["RISK_SCORE"] == score) &
    (df[orig_col] == vintage_date) &
    (df[mob_col] == mob_check)
].copy()

print("\n================ RAW DATA CHI TIẾT ================\n")
print(df_raw_group)


# ==========================================================
# 2) KẾT QUẢ AMOUNT-BASED TỪ get_actual_all_vintages_amount
# ==========================================================

actual_results = get_actual_all_vintages_amount(df)

key = (product, score, vintage_date)

mob_dict = actual_results.get(key, {})

actual_ead_vec = mob_dict.get(mob_check)

print("\n================ ACTUAL EAD TỪ LIFECYCLE ================\n")
print(actual_ead_vec)


# ==========================================================
# 3) SO SÁNH RAW VS ACTUAL LIFECYCLE
# ==========================================================

check_raw = (
    df_raw_group.groupby(state_col)[ead_col].sum()
    .reindex(BUCKETS_CANON, fill_value=0.0)
)

df_compare = pd.DataFrame({
    "RAW_SUM_EAD": check_raw,
    "ACTUAL_EAD": actual_ead_vec
})

print("\n================ SO SÁNH RAW vs ACTUAL =================\n")
print(df_compare)



================ RAW DATA CHI TIẾT ================

        CUTOFF_DATE  AGREEMENT_ID DISBURSAL_DATE  DISBURSAL_AMOUNT  DPD_EOM  \
528      2023-02-01       4128021     2023-01-01        21540000.0        0   
1024     2023-02-01       4138534     2023-01-01        21540000.0        0   
1086     2023-02-01       4139550     2023-01-01        16155000.0        0   
1737     2023-02-01       4146999     2023-01-01        21540000.0        0   
1830     2023-02-01       4147182     2023-01-01        21540000.0        0   
...             ...           ...            ...               ...      ...   
3195235  2023-02-01       4200155     2023-01-01        21540000.0        0   
3195241  2023-02-01       4193058     2023-01-01        16155000.0        0   
3195243  2023-02-01       4195416     2023-01-01        21540000.0        0   
3195245  2023-02-01       4179735     2023-01-01        10770000.0        0   
3195262  2023-02-01       4230912     2023-01-01        21540000.0        0  

In [62]:
df_raw_group.to_excel('df_raw_group.xlsx')

In [63]:
import pandas as pd

from src.config import CFG, BUCKETS_CANON
from src.rollrate.forecast import forecast_all_vintages
from src.rollrate.lifecycle import (
    get_actual_all_vintages_amount,
    combine_all_lifecycle_amount,
    lifecycle_to_long_df_amount,
    tag_forecast_rows_amount,
    add_del_metrics,
    aggregate_to_product,
    make_metric_pivot,
)

# === COHORT CẦN CHECK ===
product = "SALPIL"
score = "A"
vintage_date = pd.Timestamp("2023-01-01")
mob_check = 1

orig_col = CFG["orig_date"]
mob_col  = CFG["mob"]
state_col = CFG["state"]
ead_col  = CFG["ead"]
disp_col = CFG["disb"]


# ============================================================
# STEP 1: RAW DATA chi tiết theo hợp đồng
# ============================================================

df_raw_group = df[
    (df["PRODUCT_TYPE"] == product) &
    (df["RISK_SCORE"] == score) &
    (df[orig_col] == vintage_date)
].copy()

# Nếu muốn chỉ xem MOB = 1:
df_raw_group_mob1 = df_raw_group[df_raw_group[mob_col] == mob_check].copy()

df_raw_group.to_excel("step1_raw_full_vintage.xlsx", index=False)
df_raw_group_mob1.to_excel("step1_raw_vintage_mob1.xlsx", index=False)


# ============================================================
# STEP 2: ACTUAL LIFECYCLE (amount-based) từ hàm
# ============================================================

actual_results = get_actual_all_vintages_amount(df)
key = (product, score, vintage_date)

actual_mob_dict = actual_results.get(key, {})
actual_ead_vec_mob1 = actual_mob_dict.get(mob_check)

# Toàn bộ lifecycle actual cho cohort này
df_actual_full = pd.DataFrame(actual_mob_dict).T
df_actual_full.index.name = "MOB"

# Vector tại MOB=1
df_actual_mob1 = actual_ead_vec_mob1.to_frame(name="EAD_ACTUAL")

df_actual_full.to_excel("step2_actual_lifecycle_full.xlsx")
df_actual_mob1.to_excel("step2_actual_mob1_vector.xlsx")


# ============================================================
# STEP 3: FORECAST LIFECYCLE (amount-based)
# ============================================================

forecast_results = forecast_all_vintages(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=29,
    enable_macro=False
)

forecast_mob_dict = forecast_results.get(key, {})
df_forecast_full = pd.DataFrame(forecast_mob_dict).T
df_forecast_full.index.name = "MOB"

forecast_ead_vec_mob1 = forecast_mob_dict.get(mob_check)
if forecast_ead_vec_mob1 is not None:
    df_forecast_mob1 = forecast_ead_vec_mob1.to_frame(name="EAD_FORECAST")
else:
    df_forecast_mob1 = pd.DataFrame(columns=["EAD_FORECAST"])

df_forecast_full.to_excel("step3_forecast_lifecycle_full.xlsx")
df_forecast_mob1.to_excel("step3_forecast_mob1_vector.xlsx")


# ============================================================
# STEP 4: MERGE actual + forecast → FULL LIFECYCLE dict
# ============================================================

lifecycle = combine_all_lifecycle_amount(actual_results, forecast_results)

full_mob_dict = lifecycle.get(key, {})
df_lifecycle_full = pd.DataFrame(full_mob_dict).T
df_lifecycle_full.index.name = "MOB"

df_lifecycle_full.to_excel("step4_full_lifecycle_dict.xlsx")


# ============================================================
# STEP 5: LONG FORMAT cho toàn bộ portfolio + lọc cohort
# ============================================================

df_long = lifecycle_to_long_df_amount(lifecycle)

df_long_cohort = df_long[
    (df_long["PRODUCT_TYPE"] == product) &
    (df_long["RISK_SCORE"] == score) &
    (df_long["VINTAGE_DATE"] == vintage_date)
].sort_values("MOB")

df_long.to_excel("step5_lifecycle_long_all.xlsx", index=False)
df_long_cohort.to_excel("step5_lifecycle_long_cohort.xlsx", index=False)


# ============================================================
# STEP 6: Gắn cờ IS_FORECAST
# ============================================================

df_tagged_cohort = tag_forecast_rows_amount(df_long_cohort, df)

df_tagged_cohort.to_excel("step6_lifecycle_tagged_cohort.xlsx", index=False)


# ============================================================
# STEP 7: Thêm DEL30+, DEL60+, DEL90+ (amount & % trên DISB)
# ============================================================

df_del_cohort = add_del_metrics(df_tagged_cohort, df)

df_del_cohort.to_excel("step7_del_metrics_cohort.xlsx", index=False)


# ============================================================


TypeError: add_del_metrics() got an unexpected keyword argument 'df_tagged'

In [64]:

# STEP 8: Aggregate từ Product_Score → Product
# ============================================================

df_del_all = add_del_metrics(df_lifecycle=df_long, df_raw=df)

df_prod_agg = aggregate_to_product(df_del_all)

df_prod_cohort = df_prod_agg[
    (df_prod_agg["PRODUCT_TYPE"] == product) &
    (df_prod_agg["VINTAGE_DATE"] == vintage_date)
].sort_values("MOB")

df_prod_cohort.to_excel("step8_product_aggregated_cohort.xlsx", index=False)

# ============================================================
# STEP 9: PIVOT DEL30_PCT cho Product-level (heatmap format)
# ============================================================

pivot_del30 = make_metric_pivot(df_prod_agg, "DEL30_PCT")

pivot_del30.to_excel("step9_pivot_del30_product.xlsx")


D:\Python_code\RR_Model\src\rollrate\lifecycle.py:213: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  


In [74]:
product = "SALPIL"
vintage_date = pd.Timestamp("2023-01-01")
mob_check = 1


In [75]:
df_raw_cohort = df[
    (df["PRODUCT_TYPE"] == product) &
    (df["DISBURSAL_DATE"] == vintage_date) &
    (df["MOB"] == mob_check)
].copy()

df_raw_cohort.to_excel("step01_raw_data.xlsx", index=False)
df_raw_cohort


,CUTOFF_DATE,AGREEMENT_ID,DISBURSAL_DATE,DISBURSAL_AMOUNT,DPD_EOM,RISK_BUCKET,PRINCIPLE_OUTSTANDING,STATUS,MOB,PRODUCT_TYPE,NPA_STAGEID,DPD_GROUP,STATE_MODEL,RISK_SCORE
1,2023-02-01,4089415,2023-01-01,18309000.0,0,B0,18273533,A,1,SALPIL,REGULAR,DPD0,DPD0,B
32,2023-02-01,4095105,2023-01-01,19924500.0,0,B0,19899795,A,1,SALPIL,REGULAR,DPD0,DPD0,C
63,2023-02-01,4103952,2023-01-01,21540000.0,0,B0,21020747,A,1,SALPIL,REGULAR,DPD0,DPD0,B
94,2023-02-01,4109185,2023-01-01,21540000.0,0,B0,21297330,A,1,SALPIL,REGULAR,DPD0,DPD0,B
125,2023-02-01,4116058,2023-01-01,21540000.0,0,B0,20830723,A,1,SALPIL,REGULAR,DPD0,DPD0,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195276,2023-02-01,4206222,2023-01-01,20463000.0,18,B1,20334055,A,1,SALPIL,TB0,DPD1+,DPD1+,D
3195281,2023-02-01,4197800,2023-01-01,21540000.0,0,B0,19972566,A,1,SALPIL,REGULAR,DPD0,DPD0,B
3195283,2023-02-01,4166872,2023-01-01,12385500.0,0,B0,11549102,A,1,SALPIL,REGULAR,DPD0,DPD0,B
3195284,2023-02-01,4258417,2023-01-01,21540000.0,0,B0,21540000,A,1,SALPIL,REGULAR,DPD0,DPD0,B


In [76]:
actual_all = get_actual_all_vintages_amount(df)

actual_cohort = actual_all.get((product, None, vintage_date), None)
# nhưng actual_all dùng key = (product, score, vintage)
# nên lấy tất cả score:

actual_score_groups = {
    (p,s,v): mob_dict
    for (p,s,v), mob_dict in actual_all.items()
    if p == product and v == vintage_date
}

# xuất từng score ra excel
for (p, s, v), mob_dict in actual_score_groups.items():
    df_tmp = pd.DataFrame(mob_dict).T
    df_tmp.index.name = "MOB"
    df_tmp.to_excel(f"step02_actual_{product}_{s}_{vintage_date.date()}.xlsx")


In [77]:
forecast_all = forecast_all_vintages(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=29
)

forecast_score_groups = {
    (p,s,v): mob_dict
    for (p,s,v), mob_dict in forecast_all.items()
    if p == product and v == vintage_date
}

for (p, s, v), mob_dict in forecast_score_groups.items():
    df_tmp = pd.DataFrame(mob_dict).T
    df_tmp.index.name = "MOB"
    df_tmp.to_excel(f"step03_forecast_{product}_{s}_{vintage_date.date()}.xlsx")


In [78]:
lifecycle = combine_all_lifecycle_amount(actual_all, forecast_all)

lifecycle_score_groups = {
    (p,s,v): mob_dict
    for (p,s,v), mob_dict in lifecycle.items()
    if p == product and v == vintage_date
}

for (p, s, v), mob_dict in lifecycle_score_groups.items():
    df_tmp = pd.DataFrame(mob_dict).T
    df_tmp.index.name = "MOB"
    df_tmp.to_excel(f"step04_lifecycle_{product}_{s}_{vintage_date.date()}.xlsx")


In [79]:
df_long_tagged = tag_forecast_rows_amount(df_long, df)

df_tagged_cohort = df_long_tagged[
    (df_long_tagged["PRODUCT_TYPE"] == product) &
    (df_long_tagged["VINTAGE_DATE"] == vintage_date)
]

df_tagged_cohort.to_excel("step06_tagged.xlsx", index=False)
df_tagged_cohort


,PRODUCT_TYPE,RISK_SCORE,VINTAGE_DATE,MOB,DPD0,DPD1+,DPD30+,DPD60+,DPD90+,DPD120+,DPD180+,PREPAY,WRITEOFF,SOLDOUT,ACTUAL_MAX_MOB,IS_FORECAST
0,SALPIL,A,2023-01-01,0,5.270872e+10,3.173485e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,30,0
1,SALPIL,A,2023-01-01,1,5.014614e+10,1.089986e+09,21540000.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,30,0
2,SALPIL,A,2023-01-01,2,4.718645e+10,1.359809e+09,577088071.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,30,0
3,SALPIL,A,2023-01-01,3,4.400362e+10,1.817597e+09,791155878.0,534143803.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,30,0
4,SALPIL,A,2023-01-01,4,4.124273e+10,1.854710e+09,830096726.0,752995981.0,436883942.0,0.0,0.0,0.0,0.000000e+00,0.0,30,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3005,SALPIL,D,2023-01-01,26,4.489064e+08,1.785686e+07,42934123.0,82454247.0,713910621.0,0.0,0.0,0.0,1.111695e+10,0.0,30,0
3006,SALPIL,D,2023-01-01,27,3.878198e+08,3.318604e+07,19680334.0,26024937.0,618294427.0,0.0,0.0,0.0,1.125420e+10,0.0,30,0
3007,SALPIL,D,2023-01-01,28,3.353957e+08,1.593940e+07,31363474.0,15353108.0,483966171.0,0.0,0.0,0.0,1.136961e+10,0.0,30,0
3008,SALPIL,D,2023-01-01,29,2.841726e+08,2.040365e+07,15070973.0,28790024.0,356174158.0,0.0,0.0,0.0,1.146227e+10,0.0,30,0


In [80]:
df_del_all = add_del_metrics(
    df_lifecycle=df_long_tagged,
    df_raw=df
)

df_del_cohort = df_del_all[
    (df_del_all["PRODUCT_TYPE"] == product) &
    (df_del_all["VINTAGE_DATE"] == vintage_date)
]

df_del_cohort.to_excel("step07_del_metrics.xlsx", index=False)
df_del_cohort


,PRODUCT_TYPE,RISK_SCORE,VINTAGE_DATE,MOB,DPD0,DPD1+,DPD30+,DPD60+,DPD90+,DPD120+,...,SOLDOUT,ACTUAL_MAX_MOB,IS_FORECAST,DISB_TOTAL,DEL30_AMT,DEL60_AMT,DEL90_AMT,DEL30_PCT,DEL60_PCT,DEL90_PCT
0,SALPIL,A,2023-01-01,0,5.270872e+10,3.173485e+08,0.0,0.0,0.0,0.0,...,0.0,30,0,5.352356e+10,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000
1,SALPIL,A,2023-01-01,1,5.014614e+10,1.089986e+09,21540000.0,0.0,0.0,0.0,...,0.0,30,0,5.352356e+10,2.154000e+07,0.000000e+00,0.0,0.000402,0.000000,0.000000
2,SALPIL,A,2023-01-01,2,4.718645e+10,1.359809e+09,577088071.0,0.0,0.0,0.0,...,0.0,30,0,5.352356e+10,5.770881e+08,0.000000e+00,0.0,0.010782,0.000000,0.000000
3,SALPIL,A,2023-01-01,3,4.400362e+10,1.817597e+09,791155878.0,534143803.0,0.0,0.0,...,0.0,30,0,5.352356e+10,1.325300e+09,5.341438e+08,0.0,0.024761,0.009980,0.000000
4,SALPIL,A,2023-01-01,4,4.124273e+10,1.854710e+09,830096726.0,752995981.0,436883942.0,0.0,...,0.0,30,0,5.352356e+10,2.019977e+09,1.189880e+09,436883942.0,0.037740,0.022231,0.008162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3005,SALPIL,D,2023-01-01,26,4.489064e+08,1.785686e+07,42934123.0,82454247.0,713910621.0,0.0,...,0.0,30,0,2.031753e+10,8.392990e+08,7.963649e+08,713910621.0,0.041309,0.039196,0.035138
3006,SALPIL,D,2023-01-01,27,3.878198e+08,3.318604e+07,19680334.0,26024937.0,618294427.0,0.0,...,0.0,30,0,2.031753e+10,6.639997e+08,6.443194e+08,618294427.0,0.032681,0.031712,0.030432
3007,SALPIL,D,2023-01-01,28,3.353957e+08,1.593940e+07,31363474.0,15353108.0,483966171.0,0.0,...,0.0,30,0,2.031753e+10,5.306828e+08,4.993193e+08,483966171.0,0.026119,0.024576,0.023820
3008,SALPIL,D,2023-01-01,29,2.841726e+08,2.040365e+07,15070973.0,28790024.0,356174158.0,0.0,...,0.0,30,0,2.031753e+10,4.000352e+08,3.849642e+08,356174158.0,0.019689,0.018947,0.017530


In [81]:
df_prod_agg = aggregate_to_product(df_del_all)

df_prod_cohort = df_prod_agg[
    (df_prod_agg["PRODUCT_TYPE"] == product) &
    (df_prod_agg["VINTAGE_DATE"] == vintage_date)
].sort_values("MOB")

df_prod_cohort.to_excel("step08_product_aggregated.xlsx", index=False)
df_prod_cohort


D:\Python_code\RR_Model\src\rollrate\lifecycle.py:285: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  


,PRODUCT_TYPE,VINTAGE_DATE,MOB,DEL30_PCT,DEL60_PCT,DEL90_PCT,PRODUCT_DISB
0,SALPIL,2023-01-01,0,0.000000,0.000000,0.000000,2.803124e+11
1,SALPIL,2023-01-01,1,0.004517,0.000000,0.000000,2.803124e+11
2,SALPIL,2023-01-01,2,0.041620,0.003904,0.000000,2.803124e+11
3,SALPIL,2023-01-01,3,0.085343,0.038788,0.003367,2.803124e+11
4,SALPIL,2023-01-01,4,0.122337,0.079140,0.034650,2.803124e+11
5,SALPIL,2023-01-01,5,0.156687,0.111193,0.068368,2.803124e+11
6,SALPIL,2023-01-01,6,0.177220,0.142025,0.105138,2.803124e+11
7,SALPIL,2023-01-01,7,0.196817,0.164609,0.133323,2.803124e+11
8,SALPIL,2023-01-01,8,0.210032,0.178567,0.151061,2.803124e+11
9,SALPIL,2023-01-01,9,0.193261,0.163470,0.139942,2.803124e+11


In [82]:
df_pivot = df_prod_cohort.pivot_table(
    index="VINTAGE_DATE",
    columns="MOB",
    values="DEL30_PCT"
)

df_pivot.to_excel("step09_pivot_del30.xlsx")
df_pivot


MOB,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
VINTAGE_DATE,,,,,,,,,,,,,,,,,,,,,
2023-01-01,0.0,0.004517,0.04162,0.085343,0.122337,0.156687,0.17722,0.196817,0.210032,0.193261,...,0.097041,0.090892,0.075941,0.0608,0.051984,0.04187,0.032148,0.02507,0.018971,0.013798


In [89]:
df_lifecycle = build_full_lifecycle_amount(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=29
)


In [90]:
df_lifecycle_tagged = tag_forecast_rows_amount(
    df_lifecycle=df_lifecycle,
    df_raw=df
)


In [91]:
df_del = add_del_metrics(
    df_lifecycle=df_lifecycle_tagged,
    df_raw=df
)


In [92]:
df_del_prod = aggregate_to_product(df_del)


D:\Python_code\RR_Model\src\rollrate\lifecycle.py:285: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  


In [93]:
df_del30_pivot = df_del_prod.pivot_table(
    index="VINTAGE_DATE",
    columns="MOB",
    values="DEL30_PCT"
).fillna(0)

df_del30_pivot.index.name = "COHORT"


In [94]:
export_heatmap(df_del30_pivot, filename="DEL30P_LIFECYCLE.xlsx")


✔ Exported heatmap to DEL30P_LIFECYCLE.xlsx
